In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Реализация логистической регрессии

 

In [4]:
from sklearn.base import BaseEstimator, RegressorMixin

class LogReg(BaseEstimator, RegressorMixin):
    
    def __init__(self, batch_size=1000, num_steps=350, lr=0.05):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.lr = lr
    
    def sigmoid(self, x):
            return 1 / (1 + np.exp(-x))
    
    def count_loss(self, y, f):
            return np.mean(-y * np.log(f) - (1-y) * np.log(1-f))

    def fit(self, X, Y):
        self.losses = []
        w = np.random.randn(X.shape[1])[:, None]
        n_objects = len(X)
        for i in range(self.num_steps):
            sample_indices = np.random.randint(0, n_objects, size=self.batch_size)
            f = self.sigmoid(np.dot(X[sample_indices], w))
            loss = self.count_loss(Y[sample_indices], f)
            w -= self.lr * np.dot(X[sample_indices].T, f - Y[sample_indices]) / self.batch_size
            self.losses.append(loss)

        self.w = w
        return self

    def predict(self, X):
        prediction = self.sigmoid(X@self.w)
        return np.round(prediction)

 Я взял датасет, встроенныё в sklearn. Выбранный датасет содержит картинки цифр в пиксельном представлении.
 Начальное количество классов - 10. Так как наша логистическая регрессия должна быть бинарной, то я разделил все цифры на 2 класса : '<5' и '>=5'

In [61]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


data = load_digits()
X = data['data']
y = (data['target'] >= 5).astype(np.int32)
y = y.reshape((y.size,1))
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.77, shuffle=True)

scaler = StandardScaler()
scaler.fit(x_train)
x_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [62]:
print(X.shape)

(1797, 64)


#### Сравнение моделей

In [63]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score


own_model = LogReg().fit(x_scaled, y_train)
y_pred_1 = own_model.predict(x_test_scaled)

sklearn_model = LogisticRegression().fit(x_scaled, y_train)
y_pred_2 = sklearn_model.predict(x_test_scaled)
print('Сравение доли правильных ответов для собственной модели и модели из коробки:')
print(f'Точность собственной модели: {round(accuracy_score(y_test,y_pred_1), 4) * 100}%')
print(f'Точность модели из коробки: {round(accuracy_score(y_test,y_pred_2), 4) * 100}%')

Сравение доли правильных ответов для собственной модели и модели из коробки:
Точность собственной модели: 83.82%
Точность модели из коробки: 88.89%


По метрике Accuracy можно сказать что точность собственной модели ниже на несколько процентов. Однако при каждом новом запуске кода точность обоих моделей разниться (от 77 до 95 %) и я предположу, что это происходит из-за относительно маленького датасета для такого "сложного" предсказания. 

In [64]:
print('Веса, полученные в собственной модели по расположению пикселей:')
print(own_model.w.round(2).reshape(8, 8))


Веса, полученные в собственной модели по расположению пикселей:
[[-0.93  0.98 -0.01  0.56  0.02  1.22  0.11  0.43]
 [-1.59 -0.16 -0.14  0.66 -0.27 -0.77  0.08 -0.14]
 [-0.3   0.08  0.68 -0.53 -0.82  0.86  0.16 -0.06]
 [-0.23 -0.57  1.67  1.   -0.03  0.58 -0.49 -0.07]
 [ 0.03 -1.59  0.68  0.61 -0.23  0.12 -0.59 -2.05]
 [-0.07  0.43 -0.3   0.35  0.59  0.49  1.11 -0.38]
 [-0.02  0.21  0.14 -0.23 -1.86 -1.59 -0.31  0.01]
 [-0.09 -1.52  1.01 -0.1  -0.01  0.94 -0.58  0.04]]


In [65]:
import plotly.express as px
fig = px.scatter(y=own_model.losses, trendline='lowess', trendline_options=dict(frac=0.1) , 
                 trendline_color_override='orange', title='Зависимость значения функции потерь от "эпохи" в нашей модели')
fig.update_xaxes(title_text='epoch')
fig.update_yaxes(title_text='Logloss')
fig.show()

В данном случае график является подтверждением того, что собственная модель работает правильно, так как "потери" значительно уменьшаются с каждой эпохой.

# Реализация линейной регрессии с регуляризацией L2

In [39]:
from sklearn.base import BaseEstimator, RegressorMixin

class LinRegL2(BaseEstimator, RegressorMixin):
    
    def __init__(self, batch_size=25, num_steps=350, lr=1e-2, alpha=0.01):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.lr = lr
        self.alpha = alpha

    def fit(self, X, Y):
        w = np.random.randn(X.shape[1])[:, None]
        n_objects = len(X)
        for i in range(self.num_steps):
            sample_indices = np.random.randint(0, n_objects, size=self.batch_size)
            w +=  - self.lr * (2  * np.dot(X[sample_indices].T, np.dot(X[sample_indices], w) - Y[sample_indices])
                               + 2 * self.alpha * w) / self.batch_size

        self.w = w
        return self

    def predict(self, X):
        return X@self.w

In [40]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

#Генерация данных для тестирования
n_features = 700
n_objects = 100000

w_true = np.random.uniform(-2, 2, (n_features, 1))

X = np.random.uniform(-100, 100, (n_objects, n_features)) * np.arange(n_features)
Y = X.dot(w_true) + np.random.normal(0, 10, (n_objects, 1))

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [42]:
scaler = StandardScaler()
scaler.fit(x_train)
x_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [48]:
own_model = LinRegL2().fit(x_scaled, y_train)
y_pred = own_model.predict(x_test_scaled)
own_r2 = r2_score(y_test, y_pred)

sklearn_model = Ridge(alpha=0.01).fit(x_scaled, y_train)
y_pred = sklearn_model.predict(x_test_scaled)
sklearn_r2 = r2_score(y_test, y_pred)
    
print('R^2 in own model:', own_r2)
print('R^2 in sklearn loss:', sklearn_r2)

R^2 in own model: 0.9999101751455426
R^2 in sklearn loss: 0.9999999998151321


Точность обоих моделей очень высока (но у sklearn чуть выше). 
Для сравнения, на этом же датасете с помощью обычной линейной регрессии (без регуляризации) были получены такие результаты:

    R^2 in own model: 0.9998833465431146
    R^2 in sklearn loss: 0.9999999998181076
(С регуляризацией точность немного повысилась)